In [7]:
import pandas as pd
import regex as re
import numpy as np

chpsIdGenerator = 1     # 0 for world champ 2022


def getChpsId(dummy):
    global chpsIdGenerator
    res = 'c' + str('%03d' % chpsIdGenerator)
    chpsIdGenerator +=1
    return res

In [8]:
fileDir = r'ChampionshipList.txt'

chpsDf = pd.read_table(fileDir)

chpsDf

,Name,Region,NumOfGames,Duration,FirstGame,LastGame
0,World Championship 2022,WR,80,31:40,2022-10-07,2022-11-05
1,World Championship Play-In 2022,WR,47,31:09,2022-09-29,2022-10-04
2,EU Masters Summer 2022,WR,77,30:31,2022-08-29,2022-09-25
3,LFL Promotion 2023,FR,19,32:47,2022-09-12,2022-09-20
4,LCS Championship 2022,NaN,55,33:45,2022-08-20,2022-09-11
...,...,...,...,...,...,...
119,Prime League Spring 2022,DE,90,32:14,2022-01-11,2022-02-24
120,LCL Spring 2022,CIS,16,29:47,2022-02-12,2022-02-20
121,LCS Lock In 2022,NaN,39,31:52,2022-01-14,2022-01-30
122,Demacia Cup 2021,CN,66,31:35,2021-12-17,2021-12-26


In [9]:
typePat = r'(?P<type>[A-Z]+ )'
yearPat = r'(?P<year>\d{4})'

def getChpsTypeAndYear(strVal):
    strVal = str(strVal)
    t = re.search(pattern=typePat, string=strVal)
    y = re.search(pattern=yearPat, string=strVal)
    if t and y:
        return t.group(0).rstrip(), y.group(0)
    elif t:
        return t.group(0).rstrip(), ''
    elif y:
        return '', y.group(0)
    else:
        return '', ''


chpsDf['LeagueName'] = chpsDf.Name.map(lambda s : getChpsTypeAndYear(s)[0].rstrip())
chpsDf['Year'] = chpsDf.Name.map(lambda s : int(getChpsTypeAndYear(s)[1]) if getChpsTypeAndYear(s)[1] != '' else '')
chpsDf

,Name,Region,NumOfGames,Duration,FirstGame,LastGame,LeagueName,Year
0,World Championship 2022,WR,80,31:40,2022-10-07,2022-11-05,,2022
1,World Championship Play-In 2022,WR,47,31:09,2022-09-29,2022-10-04,,2022
2,EU Masters Summer 2022,WR,77,30:31,2022-08-29,2022-09-25,EU,2022
3,LFL Promotion 2023,FR,19,32:47,2022-09-12,2022-09-20,LFL,2023
4,LCS Championship 2022,NaN,55,33:45,2022-08-20,2022-09-11,LCS,2022
...,...,...,...,...,...,...,...,...
119,Prime League Spring 2022,DE,90,32:14,2022-01-11,2022-02-24,,2022
120,LCL Spring 2022,CIS,16,29:47,2022-02-12,2022-02-20,LCL,2022
121,LCS Lock In 2022,NaN,39,31:52,2022-01-14,2022-01-30,LCS,2022
122,Demacia Cup 2021,CN,66,31:35,2021-12-17,2021-12-26,,2021


In [10]:
regLabels = np.unique(chpsDf.LeagueName)
regLabels = regLabels[1:]
regLabels

array(['CBLOL', 'EBL', 'EU', 'GLL', 'LCK', 'LCL', 'LCO', 'LCS', 'LEC',
       'LFL', 'LJL', 'LLA', 'LPL', 'LPLOL', 'LVP', 'MSI', 'NA', 'NLC',
       'PCS', 'PG', 'TCL', 'VCS'], dtype=object)

In [11]:
isRegional = chpsDf.LeagueName.map(lambda v : v in regLabels)

chpsDf = chpsDf[isRegional]

In [12]:
def getLeagueType(strVal):
    tokens = strVal.split(' ')[1:-1]
    if len(tokens) == 0:
        return ''
    else:
        s = ''
        for t in tokens:
            s += t + ' '
        return s.rstrip()

    
chpsDf['LeagueType'] = chpsDf.Name.map(getLeagueType)
chpsDf

C:\Users\Chaconne\AppData\Local\Temp\ipykernel_34996\978912992.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chpsDf['LeagueType'] = chpsDf.Name.map(getLeagueType)


,Name,Region,NumOfGames,Duration,FirstGame,LastGame,LeagueName,Year,LeagueType
2,EU Masters Summer 2022,WR,77,30:31,2022-08-29,2022-09-25,EU,2022,Masters Summer
3,LFL Promotion 2023,FR,19,32:47,2022-09-12,2022-09-20,LFL,2023,Promotion
4,LCS Championship 2022,NaN,55,33:45,2022-08-20,2022-09-11,LCS,2022,Championship
5,LEC Summer Playoffs 2022,EUW,31,33:47,2022-08-26,2022-09-11,LEC,2022,Summer Playoffs
6,CBLOL Academy Split 2 Playoffs 2022,BR,15,32:04,2022-08-15,2022-09-10,CBLOL,2022,Academy Split 2 Playoffs
...,...,...,...,...,...,...,...,...,...
115,PG Nationals Spring 2022,IT,56,33:14,2022-01-18,2022-03-02,PG,2022,Nationals Spring
117,LPLOL Spring 2022,PT,56,31:35,2022-01-17,2022-03-02,LPLOL,2022,Spring
118,GLL Spring 2022,GR,56,31:10,2022-01-17,2022-03-01,GLL,2022,Spring
120,LCL Spring 2022,CIS,16,29:47,2022-02-12,2022-02-20,LCL,2022,Spring


In [13]:
chpsTab = pd.DataFrame({'chpsSeason': chpsDf.LeagueType, 'chpsRegion':chpsDf.LeagueName, 'chpsYear':chpsDf.Year})
chpsTab.sort_values(by='chpsRegion',inplace=True)
chpsTab.reset_index(drop=True, inplace=True)
chpsTab['chpsId'] = chpsTab.index.map(getChpsId)
chpsTab.to_csv('ChampionShip.csv')
chpsTab

,chpsSeason,chpsRegion,chpsYear,chpsId
0,Academy Split 1 Playoffs,CBLOL,2022,c001
1,Academy Split 2,CBLOL,2022,c002
2,Split 2,CBLOL,2022,c003
3,Academy Split 2 Playoffs,CBLOL,2022,c004
4,Split 2 Playoffs,CBLOL,2022,c005
...,...,...,...,...
93,Winter Playoffs,TCL,2022,c094
94,Spring,VCS,2022,c095
95,Summer Playoffs,VCS,2022,c096
96,Summer,VCS,2022,c097
